# Purchase Prediction

# Q1

In [2]:
import gzip
import random
from collections import defaultdict

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [4]:
data=list(readGz("train.json.gz"))
train_data = data[:100000]
valid_data = data[100000:200000]

In [5]:
epair =defaultdict(list)
pair=[]
businessID = set()
userID = set()
for l in data:
  user,business = l['reviewerID'],l['itemID']
  businessID.add(business)
  userID.add(user)
  epair[user].append(business)
  pair.append((user,business))   

In [6]:
count=0
userID=list(userID)
businessID=list(businessID)
non_purchase_pairs=[]
while count<100000:
    random_user=random.choice(userID)
    random_item=random.choice(businessID) 
    if random_item in epair[random_user]:
        continue
    else:
        non_purchase_pairs.append((random_user,random_item))
        count+=1

In [7]:
len(non_purchase_pairs)

100000

In [8]:
businessCount = defaultdict(int)
totalPurchases = 0

for l in train_data:
  user,business = l['reviewerID'],l['itemID']
  businessCount[business] += 1
  totalPurchases += 1

mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalPurchases/2: break

In [9]:
valid=[]
for l in valid_data:
    valid.append((l['reviewerID'],l['itemID']))

In [10]:
validation=valid+non_purchase_pairs

In [11]:
prediction=[]
for i in validation:
    if i[1] in return1:
        prediction.append(1)
    else:
        prediction.append(0)      

In [12]:
list1=[1]*100000
list0=[0]*100000
compare=list1+list0

In [13]:
get=[]
for n in range(len(prediction)):
    if prediction[n]==compare[n]:
        get.append(1)
    else:
        get.append(0)
acc=sum(get)/len(get)
acc

0.63

## The accuracy of the baseline model on the validation set I have built is 0.627585.

# Q2 
original, popular = True, now "totalPurchases/2" as purchase prediction baseline, is it the best?

In [14]:
p=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
acclist=[]
for q in range(len(p)):
    return1 = set()
    count = 0
    for ic, i in mostPopular:
      count += ic
      return1.add(i)
      if count > totalPurchases*p[q]: break
    
    return1=set(return1)
    
    pred=[]
    for i in validation:
        if i[1] in return1:
            pred.append(1)
        else:
            pred.append(0)  
        
    get=[]
    for n in range(len(pred)):
        if pred[n]==compare[n]:
            get.append(1)
        else:
            get.append(0)
    acc=sum(get)/len(get)
    acclist.append(acc)

In [15]:
acclist

[0.54135,
 0.574925,
 0.601285,
 0.620325,
 0.63,
 0.628415,
 0.61549,
 0.5918,
 0.55586]

In [16]:
import numpy as np
acclist=[]
for p in np.arange(0.4,0.6,0.01):
    return1 = set()
    count = 0
    for ic, i in mostPopular:
      count += ic
      return1.add(i)
      if count > totalPurchases*p: break
    
    return1=set(return1)
    
    pred=[]
    for i in validation:
        if i[1] in return1:
            pred.append(1)
        else:
            pred.append(0)  
        
    get=[]
    for n in range(len(pred)):
        if pred[n]==compare[n]:
            get.append(1)
        else:
            get.append(0)
    acc=sum(get)/len(get)
    acclist.append(acc)  

In [17]:
acclist.index(max(acclist))

11

In [18]:
p=np.arange(0.4,0.6,0.01)
p[13]

0.5300000000000001

## As seen in the accuracy result, when we are using a threshold of the 53th percentile of popularity(choosing the top 53%), we are going to get a higher accuracy than the 50th percentile.

# Q3
build baseline when people purchase same category of item agian, return "True"

In [5]:
def chain(iterables):  #function to spread items in lists in list
    category=[]
    for i in range(len(iterables)):
        category+=iterables[i-1]
    return category

In [7]:
user_cate=defaultdict(set)
business_cate=defaultdict(set)
categories=set()
for l in data_try:
    user,business,categories = l['reviewerID'],l['itemID'],l['categories']
    categories=set(tuple(x) for x in categories)
    user_cate[user]= user_cate[user]|categories
    business_cate[business]= business_cate[business]|categories

KeyError: 'categories'

In [22]:
user_cate[user]

{('Clothing, Shoes & Jewelry', 'Anne Klein'),
 ('Clothing, Shoes & Jewelry', 'Available for International Shipping'),
 ('Clothing, Shoes & Jewelry', 'Fashion Watches'),
 ('Clothing, Shoes & Jewelry', 'Invicta'),
 ('Clothing, Shoes & Jewelry', 'Jewelry: International Shipping Available'),
 ('Clothing, Shoes & Jewelry', 'Men', 'Watches', 'Wrist Watches'),
 ('Clothing, Shoes & Jewelry', 'New Arrivals'),
 ('Clothing, Shoes & Jewelry', 'S', 'Steve Madden'),
 ('Clothing, Shoes & Jewelry',
  'Shoes & Accessories: International Shipping Available'),
 ('Clothing, Shoes & Jewelry', 'Sport Watches'),
 ('Clothing, Shoes & Jewelry', 'Women', 'Shoes', 'Pumps'),
 ('Clothing, Shoes & Jewelry', 'Women', 'Watches', 'Wrist Watches')}

In [23]:
predict_final = []
for l in validation:
    user1,business1= l[0],l[1]
    u = user_cate[user1]
    b = business_cate[business1]
    if set(u).intersection(set(b)):
        predict_final.append(1)
    else:
        predict_final.append(0)

In [24]:
get=[]
for n in range(len(predict_final)):
    if predict_final[n]==compare[n]:
        get.append(1)
    else:
        get.append(0)
acc=sum(get)/len(get)

In [25]:
acc

0.593255

In [ ]:
predictions = open("predictions1.csv", 'w')
for l in open("pairs_Purchase.txt"):
    if l.startswith("reviewerID"):
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    u1 = user_cate[u]
    b1 = business_cate[i]
    if set(u1).intersection(set(b1)):
        predictions.write(u + '-' + i + ",1\n")
    else:
        predictions.write(u + '-' + i + ",0\n")

predictions.close()

# The accuracy is 0.592665.

# Q4 use "pairsPurchase" to test the model

In [23]:
businessCount = defaultdict(int)
totalPurchases = 0

for l in train_data:
  user,business = l['reviewerID'],l['itemID']
  businessCount[business] += 1
  totalPurchases += 1

mostPopular = [(businessCount[x], x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
  count += ic
  return1.add(i)
  if count > totalPurchases*0.53: break

In [24]:
predictions = open("predictions.csv", 'w')
for l in open("pairs_Purchase.txt"):
    if l.startswith("reviewerID"):
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    if i in return1:
        predictions.write(u + '-' + i + ",1\n")
    else:
        predictions.write(u + '-' + i + ",0\n")

predictions.close()

# Rating prediction

# Q5

In [25]:
import random
train_data1 = data[:100000]
valid_data1 = data[100000:200000]

In [26]:
userv=[]
itemv=[]
for l in valid_data1:
    userv.append(l['reviewerID'])
    itemv.append(l['itemID'])

In [27]:
pairv = [(x, y) for x, y in zip(userv,itemv)]

In [28]:
### Rating baseline: compute averages for each user, or return the global average if we've never seen the user before
allRatings = []
userRatings = defaultdict(list)
for l in train_data1:
  user,business = l['reviewerID'],l['itemID']
  allRatings.append(l['rating'])
  userRatings[user].append(l['rating'])

globalAverage = sum(allRatings) / len(allRatings)
userAverage = {}
for u in userRatings:
  userAverage[u] = sum(userRatings[u]) / len(userRatings[u])

mse=0
for l in valid_data1:
    mse += (l['rating']-globalAverage)**2
MSE=mse/len(pairv)

In [29]:
ratingv=[]
for l in valid_data1:
    ratingv.append(l['rating'])
ratingv[:10]

[5.0, 4.0, 5.0, 3.0, 5.0, 5.0, 3.0, 4.0, 2.0, 4.0]

In [30]:
globalAverage

4.232

In [31]:
MSE

1.222481119999121

## The accuracy(MSE) of the trivial predictor on the validation set is 1.222481119999121. Alpha equals to globalAverage which is 4.23151.

# Q6 rating(user; item) ' apha + beta user + beta item; lamda=1 report MSE

In [88]:
## get initial alpha betau betai
allRatings = []
userRatings = defaultdict(list)
itemRatings = defaultdict(list)
for l in train_data1:
    user,item = l['reviewerID'],l['itemID']
    allRatings.append(l['rating'])
    userRatings[user].append(l['rating'])
    itemRatings[item].append(l['rating'])

globalAverage = sum(allRatings) / len(allRatings)
userAverage = {}
for u in userRatings:
    userAverage[u] = sum(userRatings[u]) / len(userRatings[u])
itemAverage = {}
for i in itemRatings:
    itemAverage[i] = sum(itemRatings[i]) / len(itemRatings[i])

In [89]:
## define function for joint convex
def difference(averagedic, averagedic_last, compare):
    margin = 0
    for i in averagedic:
        margin += abs(averagedic[i] - averagedic_last[i])
    if margin < compare:
        return True
    else:
        return False

In [90]:
## update alpha betau betai
def convex(globalAverage, userAverage, itemAverage,lam,compare):
    end = False
    globalAverage_last = globalAverage
    userAverage_last = userAverage
    itemAverage_last = itemAverage
    while not end:
        # update globalAverage
        n = len(train_data)
        globalAverage = 0
        for l in train_data:
            user,item = l['reviewerID'],l['itemID']
            globalAverage += (l["rating"] - (userAverage_last[user]+ itemAverage_last[item]))
        globalAverage = globalAverage/n
        
        # update userAverage
        nitems = defaultdict(int)
        for user in userAverage:
            userAverage[user] = 0
        for l in train_data:
            user,item = l['reviewerID'],l['itemID']
            nitems[user] += 1
            userAverage[user] += (l["rating"] - (globalAverage+itemAverage_last[item]))
        for user in userAverage:
            userAverage[user] = userAverage[user]/(lam + abs(nitems[user]))
        
        # update businessAverage
        nuser = defaultdict(int)
        for item in itemAverage:
            itemAverage[item] = 0
        for l in train_data:
            user,item = l['reviewerID'],l['itemID']
            nuser[item] += 1
            itemAverage[item] += (l["rating"] - (globalAverage + userAverage[user]))
        for item in itemAverage:
            itemAverage[item] = itemAverage[item]/(lam + abs(nuser[item]))
        
        #print ((globalAverage - globalAverage_last)**2)
        #compare=0.01
        if abs(globalAverage - globalAverage_last)<compare and difference(userAverage, userAverage_last, compare) and difference(itemAverage, itemAverage_last, compare):
            end = True
        else:
            globalAverage_last = globalAverage
            userAverage_last = userAverage
            itemAverage_last = itemAverage
    return globalAverage, userAverage, itemAverage

In [91]:
final_globalAverage,final_userAverage, final_itemAverage=convex(globalAverage,userAverage, itemAverage,1,0.00001)

In [92]:
final_globalAverage

4.225832905994302

In [96]:
##report mse on validation set
def cal_mse(final_globalAverage, final_userAverage, final_itemAverage):
    mse = 0.0
    userlist = []
    itemlist = []
    for i in final_userAverage:
        userlist.append(i)
    for j in final_itemAverage:
        itemlist.append(j)
    for l in valid_data:
        user,item = l['reviewerID'],l['itemID']
        if (user in userlist) and (item in itemlist):
            mse += (l["rating"]-final_globalAverage-final_userAverage[user]-final_itemAverage[item])**2
        elif (user in userlist) and (item not in itemlist):
            mse += (l["rating"]-final_globalAverage-final_userAverage[user])**2
        elif (user not in userlist) and (item in itemlist):
            mse += (l["rating"]-final_globalAverage-final_itemAverage[item])**2
        else:
            mse += (l["rating"]-final_globalAverage)**2
    mse /= 100000
    print("mean square error on validation data is " +str(mse))

In [97]:
cal_mse(final_globalAverage, final_userAverage, final_itemAverage)

mean square error on validation data is 1.2811186591058816


# Q7 report use&item ID with largest/ lowest value of beta

In [98]:
inverse_user = [(value, key) for key, value in final_userAverage.items()]
print('The user ID that have the largest values of beta is ',max(inverse_user)[1],',the beta value is ',max(inverse_user)[0])

The user ID that have the largest values of beta is  U495776285 ,the beta value is  1.4867483773308852


In [99]:
inverse_user = [(value, key) for key, value in final_userAverage.items()]
print('The user ID that have the smallest values of beta is ',min(inverse_user)[1],',the beta value is ',min(inverse_user)[0])

The user ID that have the smallest values of beta is  U204516481 ,the beta value is  -2.5517077142613376


In [100]:
inverse_item = [(value, key) for key, value in final_itemAverage.items()]
print('The item ID that have the largest values of beta is ',max(inverse_item)[1],',the beta value is ',max(inverse_item)[0])

The item ID that have the largest values of beta is  I809804570 ,the beta value is  1.2700486631942407


In [101]:
inverse_item = [(value, key) for key, value in final_itemAverage.items()]
print('The item ID that have the smallest values of beta is ',min(inverse_item)[1],',the beta value is ',min(inverse_item)[0])

The item ID that have the smallest values of beta is  I511389419 ,the beta value is  -2.5756646326290698


# Q8 find a better lamda for it

In [54]:
lam=[0,0.01,0.1,1,100]
for m in lam:  
    final_globalAverage,final_userAverage, final_itemAverage=convex(globalAverage,userAverage, itemAverage,m,0.00001)
    cal_mse(final_globalAverage, final_userAverage, final_itemAverage)

mean square error on validation data is 1.8621883545995428
mean square error on validation data is 1.8988131471912073
mean square error on validation data is 1.7488630442796071
mean square error on validation data is 1.2811186589872874
mean square error on validation data is 1.1998248180264501


In [82]:
lam=[5,6,6.5,7]
for m in lam:  
    final_globalAverage,final_userAverage, final_itemAverage=convex(globalAverage,userAverage, itemAverage,m,0.00001)
    cal_mse(final_globalAverage, final_userAverage, final_itemAverage)

mean square error on validation data is 1.1398956062804486
mean square error on validation data is 1.1379239648287354
mean square error on validation data is 1.1376840308598197
mean square error on validation data is 1.1377680417270668


In [83]:
lam=[6.3,6.4]
for m in lam:  
    final_globalAverage,final_userAverage, final_itemAverage=convex(globalAverage,userAverage, itemAverage,m,0.00001)
    cal_mse(final_globalAverage, final_userAverage, final_itemAverage)

mean square error on validation data is 1.1377354027244155
mean square error on validation data is 1.1377040237193359


In [102]:
lam=[6.6]
for m in lam:  
    final_globalAverage,final_userAverage, final_itemAverage=convex(globalAverage,userAverage, itemAverage,m,0.00001)
    cal_mse(final_globalAverage, final_userAverage, final_itemAverage)

mean square error on validation data is 1.1376780224542182


# The best lamda I can find is 6.6 and the MSE of this model is 1.1376780201574763.

In [103]:
## use the test set to generate prediction 
## get initial alpha betau betai
allRatings = []
userRatings = defaultdict(list)
itemRatings = defaultdict(list)
for l in train_data:
    user,item = l['reviewerID'],l['itemID']
    allRatings.append(l['rating'])
    userRatings[user].append(l['rating'])
    itemRatings[item].append(l['rating'])

globalAverage = sum(allRatings) / len(allRatings)
userAverage = {}
for u in userRatings:
    userAverage[u] = sum(userRatings[u]) / len(userRatings[u])
itemAverage = {}
for i in itemRatings:
    itemAverage[i] = sum(itemRatings[i]) / len(itemRatings[i])

In [104]:
test_globalAverage,test_userAverage, test_itemAverage=convex(globalAverage,userAverage, itemAverage,6.6,0.00001)

In [106]:
##report mse on test set
mse = 0.0
userlist = []
itemlist = []
for i in test_userAverage:
    userlist.append(i)
for j in test_itemAverage:
    itemlist.append(j)
predictions = open("predictionsq8.csv", 'w')
for l in open("pairs_Rating.txt"):
    if l.startswith("reviewerID"):
        predictions.write(l)
        continue
    u,i = l.strip().split('-')    
    if (u in userlist) and (i in itemlist):
        rating=test_globalAverage+test_userAverage[u]+test_itemAverage[i]
        predictions.write(u + '-' + i + ","+str(rating)+"\n")
    elif (u in userlist) and (i not in itemlist):
        rating=test_globalAverage+test_userAverage[u]
        predictions.write(u + '-' + i + ","+str(rating)+"\n")
    elif (u not in userlist) and (i in itemlist):
        rating=test_globalAverage+test_itemAverage[i]
        predictions.write(u + '-' + i + ","+str(rating)+"\n")
    else:
        rating=test_globalAverage
        predictions.write(u + '-' + i + ","+str(rating)+"\n")
predictions.close()   